In [2]:
import geopandas as gpd
from scipy.spatial.distance import cdist
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

Read data

In [3]:
cook_centroids = gpd.read_file('data/cook_centroids_all.shp')
n1 = cook_centroids.shape[0]
cook_centroids.columns, n1

(Index(['GISJOIN', 'Lon_w', 'Lat_w', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID',
        'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
        'INTPTLON', 'Shape_Leng', 'Shape_Area', 'ORIG_FID', 'YEAR', 'STUSAB',
        'REGIONA', 'DIVISIONA', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA',
        'COUSUBA', 'PLACEA', 'TRACTA', 'BLKGRP', 'CONCITA', 'AIANHHA',
        'RES_ONLYA', 'TRUSTA', 'AIHHTLI', 'AITSA', 'ANRCA', 'CBSAA', 'CSAA',
        'METDIVA', 'NECTAA', 'CNECTAA', 'NECTADIVA', 'UAA', 'CDCURRA', 'SLDUA',
        'SLDLA', 'ZCTA5A', 'SUBMCDA', 'SDELMA', 'SDSECA', 'SDUNIA', 'PCI',
        'PUMAA', 'GEO_ID', 'BTTRA', 'BTBG', 'TL_GEO_ID', 'NAME_E', 'NAME_M',
        'S1', 'S2', 'A1', 'A2', 'A3', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7',
        'HHI1', 'HHI2', 'HHI3', 'Total_Pop', 'Total_HH', 'geometry'],
       dtype='object'),
 1331)

User-defined parameters

In [4]:
p = 100
s = 1000
t = 2

Distance matrix

In [5]:
points = cook_centroids['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist()
D = cdist(points, points, metric='euclidean')
D, D.shape

(array([[    0.        ,  1082.87422978,   661.44521516, ...,
         19026.12158013, 19802.36119962, 26772.50604907],
        [ 1082.87422978,     0.        ,   671.36097906, ...,
         18306.97279514, 18782.33657054, 26010.24087613],
        [  661.44521516,   671.36097906,     0.        , ...,
         18377.69148671, 19431.02947413, 26118.43883191],
        ...,
        [19026.12158013, 18306.97279514, 18377.69148671, ...,
             0.        , 21225.23955835,  7820.49208496],
        [19802.36119962, 18782.33657054, 19431.02947413, ...,
         21225.23955835,     0.        , 25218.70463641],
        [26772.50604907, 26010.24087613, 26118.43883191, ...,
          7820.49208496, 25218.70463641,     0.        ]]),
 (1331, 1331))

Coverage

In [8]:
# discrete
A = np.zeros((n1, n1))
for i in range(n1):
    for j in range(n1):
        if D[i, j] <= s:
            A[i, j] = 1
A, A.shape

(array([[1., 0., 1., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 (1331, 1331))

In [6]:
# continuous
A = np.zeros((n1, n1))
for i in range(n1):
    for j in range(n1):
        A[i, j] = math.exp(-t * D[i, j] / s)
A, A.shape

(array([[1.00000000e+00, 1.14664081e-01, 2.66364281e-01, ...,
         2.97934483e-17, 6.30794074e-18, 5.56796777e-24],
        [1.14664081e-01, 1.00000000e+00, 2.61133905e-01, ...,
         1.25535186e-16, 4.85142967e-17, 2.55735977e-23],
        [2.66364281e-01, 2.61133905e-01, 1.00000000e+00, ...,
         1.08978291e-16, 1.32562975e-17, 2.05973941e-23],
        ...,
        [2.97934483e-17, 1.25535186e-16, 1.08978291e-16, ...,
         1.00000000e+00, 3.66430121e-19, 1.61141308e-07],
        [6.30794074e-18, 4.85142967e-17, 1.32562975e-17, ...,
         3.66430121e-19, 1.00000000e+00, 1.24540748e-22],
        [5.56796777e-24, 2.55735977e-23, 2.05973941e-23, ...,
         1.61141308e-07, 1.24540748e-22, 1.00000000e+00]]),
 (1331, 1331))

Demand

In [7]:
# sex
# groups = ['S1', 'S2']
groups = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6']
# groups = ['S1', 'S2']
n2 = len(groups)
W = cook_centroids[groups].to_numpy()

MCLP

In [8]:
class MCLP(ElementwiseProblem):
    def __init__(self, w, a, p):
        super().__init__(n_var=a.shape[1], n_obj=1, n_ieq_constr=1, xl=0, xu=1, vtype=bool)
        self.w = w
        self.a = a
        self.p = p

    def _evaluate(self, x, out, *args, **kwargs):
        y = np.max(x * self.a, axis=1)

        # Objective 1
        obj1 = np.sum(np.sum(y * self.w.T, axis=1))

        constr = np.sum(x) - self.p  # Constraint on the total number of facilities

        out["F"] = [-obj1]
        out["G"] = [constr]

problem = MCLP(W, A, p)

algorithm = GA(
    pop_size=100,
    sampling=BinaryRandomSampling(),
    crossover=TwoPointCrossover(),
    mutation=BitflipMutation(),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 500)

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

F = [res.F]

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  5.160000E+02 |  5.641100E+02 |             - |             -
     2 |      200 |  5.160000E+02 |  5.429700E+02 |             - |             -
     3 |      300 |  5.010000E+02 |  5.299400E+02 |             - |             -
     4 |      400 |  4.860000E+02 |  5.197200E+02 |             - |             -
     5 |      500 |  4.810000E+02 |  5.101600E+02 |             - |             -
     6 |      600 |  4.800000E+02 |  5.014700E+02 |             - |             -
     7 |      700 |  4.680000E+02 |  4.922500E+02 |             - |             -
     8 |      800 |  4.610000E+02 |  4.821800E+02 |             - |             -
     9 |      900 |  4.470000E+02 |  4.717200E+02 |             - |             -
    10 |     1000 |  4.430000E+02 |  4.625400E+02 |             - |             -
    11 |     1100 |  4.320000E+02 |  4.538700E+02 |             - |             -
    12 |     120

In [11]:
n_evals = [algo.evaluator.n_eval for algo in res.history]
hist_F = []
hist_cv = []
hist_cv_avg = []

for algo in res.history:
    opt = algo.opt
    hist_cv.append(opt.get("CV").min())
    hist_cv_avg.append(algo.pop.get("CV").mean())
    feas = np.where(opt.get("feasible"))[0]
    hist_F.append(opt.get("F")[feas])

k = np.where(np.array(hist_cv_avg) <= 0.0)[0].min()
print(f"Whole population feasible in Generation {k} after {n_evals[k]} evaluations.")
res.exec_time

Whole population feasible in Generation 281 after 28200 evaluations.


262.0638897418976

Inequality

In [15]:
X_mclp = pickle.load(open('data/sols/X_cov' + str(2) + '.pickle', "rb"))
x = X_mclp
y = np.max(x * A, axis=1)

# relative range
u_k = np.sum(y * W.T, axis=1) / np.sum(W, axis=0)
u_bar = np.mean(u_k)
e1 = (np.max(u_k) - np.min(u_k)) / u_bar

# variance
e2 = np.var(u_k)

# theil index
e3 = 1 / W.shape[1] * np.sum(u_k / u_bar * np.log(u_k / u_bar))

0.016510910876108238

In [ ]:
F = res.F
plt.figure(figsize=(7, 5))
plt.scatter(-F[0], 0, s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()